In [25]:
# Import packages
import numpy as np
import numpy.ma as ma
#import matplotlib.pyplot as plt
#import pandas as pd
#import math
#import xarray as xr

In [29]:
def covariance_triple_point_collocation(data1, data2, data3, flag):
# Basic Triple Point Collocation Analysis for any combination of datasets
# Input datasets must be 2D with dimensions 1xN, Mx1, or MXN 
# All datasets must have uniform dimensions 
# Flag: 1 = exclude large dataset differences abs(>5PSU) from analysis, 0=include all dataset differences regardless of size

# J. Anderson (janderson@esr.org) based on Stoffelen 1998 & Gruber et al 2016 Covariance Notation 
    
# Error check input data for uniform dimensions and valid flag value
# Python automaticaly checks that the correct numbe of imputs were given

    num = ((data1.shape == data2.shape)+(data1.shape == data3.shape)+(data2.shape == data3.shape))
    
    if num != 3:
        raise Exception('Input datasets must have uniform dimensions')

    if np.isin(flag,[0,1]) == False:
        raise Exception('Flag input must be 1 or 0')
        
    print('hello')    
    print(np.shape(data1))
    # Flatten data prior to triple point collocation analysis
    data1 = np.ravel(data1)
    data2 = np.ravel(data2)
    data3 = np.ravel(data3)
    print(np.shape(data1))
        
    # Prepare data for triple point collocation
    # If flag = 1, remove data where absolute value of differences > 5 PSU

    if flag == 1:
        
        #'Data1 - Data3'
        ds_d1_d3 = data1-data3; # Calculate the difference between Data1 & Data3
        tempLarge = np.where(np.absolute(ds_d1_d3)>5)
        data1[tempLarge]='NaN'
        data3[tempLarge]='NaN'
        del tempLarge, ds_d1_d3
        
        #'Data2 - Data3'
        ds_d2_d3 = data2-data3; # Calculate the difference between Data1 & Data3
        tempLarge = np.where(np.absolute(ds_d2_d3)>5)
        data2[tempLarge]='NaN'
        data3[tempLarge]='NaN'
        del tempLarge, ds_d2_d3

        #'Data1 - Data2'
        ds_d1_d2 = data1-data2; # Calculate the difference between Data1 & Data3
        tempLarge = np.where(np.absolute(ds_d1_d2)>5)
        data1[tempLarge]='NaN'
        data2[tempLarge]='NaN'
        del tempLarge, ds_d1_d2
    
    # Triple point collocation cannot be calculated if one dataset has a NaN. 
    # Make location of NaNs uniform for all datasets. 
    data2[np.argwhere(np.isnan(data1))]='NaN'
    data3[np.argwhere(np.isnan(data1))]='NaN'

    data1[np.argwhere(np.isnan(data2))]='NaN'
    data3[np.argwhere(np.isnan(data2))]='NaN'

    data1[np.argwhere(np.isnan(data3))]='NaN'
    data2[np.argwhere(np.isnan(data3))]='NaN'
    
    # Start triple point collocation
    # Calculate variance of each dataset
    var_d1 = np.nanvar(data1,ddof=1) # Variance computed on flattened variance
    var_d2 = np.nanvar(data2,ddof=1)
    var_d3 = np.nanvar(data3,ddof=1)
    #print('test')
    print(var_d1)
    print(var_d2)
    print(var_d3)

    # Calculate the covariance of Data1 and Data3
    covar_d1_d3 = np.diagonal(ma.cov(ma.array(data1, mask=np.isnan(data1)), ma.array(data3, mask = np.isnan(data3)),ddof=1),offset=1) # covariance of data1 and data3 is off diagonal element, masked to ignore nans

    # Calculate the covariance of Data2 and Data3
    covar_d2_d3 = np.diagonal(ma.cov(ma.array(data2, mask=np.isnan(data2)), ma.array(data3, mask = np.isnan(data3)),ddof=1),offset=1) # covariance of data2 and data3 is off diagonal element, masked to ignore nans

    # Calculate the covariance of Data1 and Data2
    covar_d1_d2 = np.diagonal(ma.cov(ma.array(data1, mask=np.isnan(data1)), ma.array(data2, mask = np.isnan(data2)),ddof=1),offset=1) # covariance of data1 and data2 is off diagonal element, masked to ignore nans
    
    print(covar_d1_d3)
    print(covar_d2_d3)
    print(covar_d1_d2)
    
    # Calculate the unscaled error variances for each dataset
    # Covariance is symmetrical, so cov(d1,d2)==cov(d2,d1)
    errorvar_d1 = (var_d1-((covar_d1_d2*covar_d1_d3)/covar_d2_d3))
    errorvar_d2 = (var_d2-((covar_d1_d2*covar_d2_d3)/covar_d1_d3))
    errorvar_d3 = (var_d3-((covar_d1_d3*covar_d2_d3)/covar_d1_d2))
    #print(errorvar_d1)
    #print(errorvar_d2)
    #print(errorvar_d3)
    
    # Determine rescaling parameters if desired
    # Rescale to d1
    #rescale_d2 = (covar_d1_d3/covar_d2_d3)
    #rescale_d3 = (covar_d1_d2/covar_d2_d3)
    #print('rescale')
    #print(rescale_d2)
    #print(rescale_d3)

    #errorvar_d1 = errorvar_d1
    #errorvar_d2 = errorvar_d2*(rescale_d2**2)
    #errorvar_d3 = errorvar_d3*(rescale_d3**2)
    #print(errorvar_d1)
    #print(errorvar_d2)
    #print(errorvar_d3)

    # Calculate RMSD from the unscaled error variances
    rmsd_d1 = np.sqrt(errorvar_d1);
    rmsd_d2 = np.sqrt(errorvar_d2);
    rmsd_d3 = np.sqrt(errorvar_d3);
    print(rmsd_d1)
    print(rmsd_d2)
    print(rmsd_d3)
    
    # NumPy sqrt only returns sqrts for positive values, replace masked, invalid numbers with nans
    rmsd_d1 = rmsd_d1.filled(np.nan)
    rmsd_d2 = rmsd_d2.filled(np.nan)
    rmsd_d3 = rmsd_d3.filled(np.nan)
    print(rmsd_d1)
    print(rmsd_d2)
    print(rmsd_d3)
    
    # It is possible to have negative unscaled variances which means imaginarrmsd. Only keep positive variances, real rmsd.
    # Replace imaginary numbers with NaN
    #rmsd_d1(imag(rmsd_d1)~=0) = NaN; 
    #rmsd_d2(imag(rmsd_d2)~=0) = NaN; 
    #rmsd_d3(imag(rmsd_d3)~=0) = NaN;

    #rmsd_d1 = real(rmsd_d1);
    #rmsd_d2 = real(rmsd_d2);
    #rmsd_d3 = real(rmsd_d3);
    print(np.shape(rmsd_d1))
    return [rmsd_d1,rmsd_d2,rmsd_d3]

In [30]:
data1 = np.array([1,2,3,2,15.0])
data2 = np.array([2.1,2.3,2,5,5])
data3 = np.array([3.0,3,4,8,8])
flag = np.array([1])

[rmsd_d1,rmsd_d2,rmsd_d3]=covariance_triple_point_collocation(data1, data2, data3, flag)

print(rmsd_d1)
print(rmsd_d2)
print(rmsd_d3)

hello
(5,)
(5,)
1.0
0.023333333333333303
0.33333333333333337
[0.5]
[-0.06666666666666665]
[-0.050000000000000044]
[0.7905694150420945]
[0.12909944487358044]
[--]
[0.79056942]
[0.12909944]
[nan]
(1,)
[0.79056942]
[0.12909944]
[nan]


/var/folders/5w/_z_sz3116qg9bl1d978q455c0000gn/T/ipykernel_70511/2247327119.py:114: RuntimeWarning: invalid value encountered in sqrt
  rmsd_d3 = np.sqrt(errorvar_d3);


In [31]:
data1 = np.array([[1,2,3,2,15.0],[1,5,3,6,15.0]])
data2 = np.array([[2.1,2.3,2,5,5],[2.7,2.3,3.3,5,5]])
data3 = np.array([[3.0,3,4,8,8],[3.3,3,4,8.5,8.9]])
testdata = np.ravel(data1)
print(testdata)
testdata2 = np.ravel(data2)
print(testdata2)
flag = np.array([1])

[rmsd_d1,rmsd_d2,rmsd_d3]=covariance_triple_point_collocation(data1, data2, data3, flag)

print(rmsd_d1)
print(rmsd_d2)
print(rmsd_d3)

[ 1.  2.  3.  2. 15.  1.  5.  3.  6. 15.]
[2.1 2.3 2.  5.  5.  2.7 2.3 3.3 5.  5. ]
hello
(2, 5)
(10,)
3.6666666666666665
1.1214285714285714
3.9414285714285717
[2.65]
[1.9430952380952382]
[1.2833333333333332]
[1.3843598636774657]
[0.424773780147373]
[--]
[1.38435986]
[0.42477378]
[nan]
(1,)
[1.38435986]
[0.42477378]
[nan]


/var/folders/5w/_z_sz3116qg9bl1d978q455c0000gn/T/ipykernel_70511/2247327119.py:114: RuntimeWarning: invalid value encountered in sqrt
  rmsd_d3 = np.sqrt(errorvar_d3);
